In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import Sequence
from sklearn.metrics import accuracy_score, classification_report

In [2]:
def count_samples(npz_path):
    with np.load(npz_path, allow_pickle=True, mmap_mode='r') as data:
        # Access the array's shape without loading the data
        for key in data.files:
            if key == 'X' or key == 'Y':
                return len(data[key])
    return 0

In [3]:
class UltraMemoryGenerator(tf.keras.utils.Sequence):
    def __init__(self, npz_path, batch_size=32, num_samples=None):
        self.npz_path = npz_path
        self.batch_size = batch_size
        self.num_samples = num_samples if num_samples else count_samples(npz_path)
        self.current_indices = np.arange(self.num_samples)
        
    def __len__(self):
        return int(np.ceil(self.num_samples / self.batch_size))
    
    def __getitem__(self, idx):
        batch_indices = self.current_indices[idx*self.batch_size:(idx+1)*self.batch_size]
        x_batch = []
        y_batch = []
        
        with np.load(self.npz_path, allow_pickle=True, mmap_mode='r') as data:
            for i in batch_indices:
                # Load one sample at a time
                x = data['X'][i].astype('float32') / 255.0
                if len(x.shape) == 2:  # Add channel dimension if missing
                    x = np.expand_dims(x, axis=-1)
                x_batch.append(x)
                y_batch.append(data['Y'][i])
        
        return np.array(x_batch), np.array(y_batch)

In [4]:
train_gen = UltraMemoryGenerator("train.npz", batch_size=4)  # Reduced batch size
test_gen = UltraMemoryGenerator("test.npz", batch_size=4)
val_gen = UltraMemoryGenerator("val.npz", batch_size=4)


In [19]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation="relu",input_shape=(224,224,1),kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(5,activation='softmax'))

c:\Users\ASUS\.conda\envs\py3-TF2.0\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.compile(optimizer='rmsprop',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [6]:
history = model.fit(train_gen, epochs=3, 
                    validation_data=val_gen,verbose=1)


NameError: name 'train_gen' is not defined

In [7]:
train=np.load('trainl.npz',allow_pickle=True)
test=np.load('testl.npz',allow_pickle=True)
val=np.load('vall.npz',allow_pickle=True)

In [21]:
history = model.fit(train['X'],train['Y'],epochs=5,validation_data=(val['X'],val['Y']),verbose='auto')

Epoch 1/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 703ms/step - accuracy: 0.4330 - loss: 4.3449 - val_accuracy: 0.8933 - val_loss: 0.3253
Epoch 2/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 15s 685ms/step - accuracy: 0.9261 - loss: 0.2072 - val_accuracy: 0.9267 - val_loss: 0.2455
Epoch 3/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 14s 656ms/step - accuracy: 0.9571 - loss: 0.1056 - val_accuracy: 0.9533 - val_loss: 0.1337
Epoch 4/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 15s 683ms/step - accuracy: 0.9815 - loss: 0.0499 - val_accuracy: 0.9667 - val_loss: 0.1264
Epoch 5/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 15s 698ms/step - accuracy: 0.9821 - loss: 0.0363 - val_accuracy: 0.9667 - val_loss: 0.1484


In [23]:
y_pred = model.predict(test['X'])

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 220ms/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [25]:
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(test['Y'], axis=1)
accuracy = accuracy_score(y_true_classes, y_pred_classes)


In [26]:
print(accuracy)

0.9533333333333334
